# Step 13: Human-in-the-Loop (HITL) with Function Tool Approvals

> ⚠️ **Coming Soon**: This lab will be added once the Microsoft Agent Framework (MAF) version is released that supports the `approval_mode` parameter.

📚 **Official Documentation**: [Function Tools with Human-in-the-Loop Approvals](https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/function-tools-approvals?pivots=programming-language-python)

## Overview

This tutorial demonstrates how to use function tools that require **human approval** before execution. This is known as the **Human-in-the-Loop (HITL)** pattern.

When agents require user input—for example, to approve a function call—the agent run completes with a response indicating what input is required, instead of returning a final answer. The caller is then responsible for:

1. Getting the required input from the user
2. Passing it back to the agent as part of a new agent run

## How It Works

### 1. Define Functions with Approval Mode

When using functions, you can indicate whether each function requires human approval before execution by setting the `approval_mode` parameter to `"always_require"`:

```python
from typing import Annotated
from agent_framework import ai_function

# Standard function - executes automatically
@ai_function
def get_weather(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get the current weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

# Function requiring approval - pauses for human confirmation
@ai_function(approval_mode="always_require")
def get_weather_detail(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get detailed weather information for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C, humidity 88%."
```

### 2. Create Agent with Approval-Required Tools

```python
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIResponsesClient

async with ChatAgent(
    chat_client=OpenAIResponsesClient(),
    name="WeatherAgent",
    instructions="You are a helpful weather assistant.",
    tools=[get_weather, get_weather_detail],
) as agent:
    # Agent is ready to use
```

### 3. Check for User Input Requests

When a function requires approval, the agent responds with a **user input request** instead of executing the function:

```python
result = await agent.run("What is the detailed weather like in Amsterdam?")

if result.user_input_requests:
    for user_input_needed in result.user_input_requests:
        print(f"Function: {user_input_needed.function_call.name}")
        print(f"Arguments: {user_input_needed.function_call.arguments}")
```

### 4. Provide Approval and Continue

Once the user decides to approve or reject, create a response and pass it back to the agent:

```python
from agent_framework import ChatMessage, Role

# Get user approval (True = approve, False = reject)
user_approval = True

# Create the approval response
approval_message = ChatMessage(
    role=Role.USER, 
    contents=[user_input_needed.create_response(user_approval)]
)

# Continue the conversation with the approval
final_result = await agent.run([
    "What is the detailed weather like in Amsterdam?",
    ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
    approval_message
])
print(final_result.text)
```

### 5. Handling Multiple Approvals in a Loop

For complex scenarios with multiple function calls requiring approval:

```python
async def handle_approvals(query: str, agent) -> str:
    """Handle function call approvals in a loop."""
    current_input = query

    while True:
        result = await agent.run(current_input)

        if not result.user_input_requests:
            return result.text  # No more approvals needed

        new_inputs = [query]
        for user_input_needed in result.user_input_requests:
            print(f"Approval needed for: {user_input_needed.function_call.name}")
            
            # Add assistant message with approval request
            new_inputs.append(ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]))
            
            # Get user approval (interactive in real app)
            user_approval = True
            
            # Add user's approval response
            new_inputs.append(
                ChatMessage(role=Role.USER, contents=[user_input_needed.create_response(user_approval)])
            )

        current_input = new_inputs

# Usage
result_text = await handle_approvals("Get detailed weather for Seattle and Portland", agent)
print(result_text)
```

## Key Concepts

| Concept | Description |
|---------|-------------|
| **approval_mode** | Set to `"always_require"` on `@ai_function` to require human approval |
| **user_input_requests** | List of pending approval requests in the agent response |
| **function_call** | Contains the function `name` and `arguments` for review |
| **create_response()** | Method to approve (`True`) or reject (`False`) a function call |

## Use Cases

| Scenario | Why HITL? |
|----------|----------|
| **Financial Transactions** | Approve before transferring funds |
| **Email/Communication** | Review before sending messages |
| **Data Modifications** | Confirm before deleting or updating records |
| **External API Calls** | Authorize calls to third-party services |
| **Sensitive Operations** | Any action with irreversible consequences |

## 🚧 Lab Status: Pending Release

> **Note:** The hands-on code cells will be added once the Microsoft Agent Framework (MAF) version is released that supports the `approval_mode` parameter.
>
> The `@ai_function(approval_mode="always_require")` feature is currently in development and not yet available in the published pip package.

## Next Steps

Once the feature is released, this notebook will include:

1. **Setup**: Configure an agent with approval-required tools
2. **Interactive Demo**: Build a UI for approving/rejecting function calls
3. **Multi-Agent Workflow**: Integrate HITL into complex workflows
4. **Best Practices**: Error handling, timeouts, and audit logging

---

📚 **Official Documentation**: [Function Tools with Human-in-the-Loop Approvals](https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/function-tools-approvals?pivots=programming-language-python)